In [428]:
!rm -r sliced_imgs

In [414]:
! pip install fast_slic

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [415]:
from fast_slic import Slic

In [416]:
import torch
import torch.nn.functional as F
from torch import nn

In [417]:
import torch
from torch import nn

lrn = nn.LocalResponseNorm(2)


def conv_unit(convs,pool,x, nrm=False):

    for conv in convs:
        x = conv(x)
    if nrm:
        x = lrn(x)
    x = F.relu(x)
    return pool(x)

class WvTNet3000(nn.Module):

    def __init__(self, H, W):
        
        super(WvTNet3000, self).__init__()
        self.conv1 = nn.Conv2d(3,16,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        
        self.conv2 = nn.Conv2d(17,32,3,padding=1)
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.conv3 = nn.Conv2d(33,63,3,padding=1)
        self.pool3 = nn.MaxPool2d(2,2) 
        
        self.conv4_0 = nn.Conv2d(64,64,3,padding=1)
        self.conv4_1 = nn.Conv2d(64,64,3,padding=1)
        self.pool4 = nn.AvgPool2d(2,2) 
        
        
        self.fc4 = nn.Linear(H*W//2**8*64, 2048)
        self.dropout4 = nn.Dropout(p=0.3)
        self.fc5 = nn.Linear(2048, 100)
        self.fc6 = nn.Linear(100, 2)
        self.H = H
        self.W = W

       
        
    def forward(self, roi):
        img, (wv1, wv2, wv3) = roi
        
        #x = torch.cat((img,wv1.unsqueeze(1)),1)
        x = lrn(F.relu(self.conv1(img)))
        
        x = self.pool1(x)
        
        x = torch.cat((x,wv1.unsqueeze(1)),1)
        x = conv_unit([self.conv2],self.pool2,x)

        x = torch.cat((x,wv2.unsqueeze(1)),1)
        x = conv_unit([self.conv3],self.pool3,x)
        
        x = torch.cat((x,wv3.unsqueeze(1)),1)
        x = conv_unit([self.conv4_0, self.conv4_1],self.pool4,x)
        
        x = x.view(-1, self.H*self.W//2**8*64)  
        x = torch.sigmoid(self.fc6(self.fc5(self.dropout4(self.fc4(x)))))

        return x



In [418]:
from glob import glob

In [419]:
import json
import numpy as np
import os
import cv2 as cv
from tqdm import tnrange
from tqdm.notebook import tqdm
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

In [420]:
def get_augmentation():
    train_transform = [
        
        albu.HorizontalFlip(p=0.5),
        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=(-20,20), shift_limit=0.1, p=1, border_mode=0),
        #albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        #albu.RandomCrop(height=320, width=320, always_apply=True),
        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),
        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),
        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),
        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
        albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]
    return albu.Compose(train_transform)

def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(416, 544)
    ]
    return albu.Compose(test_transform)


def to_tensor(x):
    return np.moveaxis(x, -1,0).astype('float32')


def get_preproc(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor),
    ]
    return albu.Compose(_transform)

In [421]:
import pywt

In [423]:
def get_wv_feature_pyr(img, mother= 'db2', channels=cv.COLOR_BGR2GRAY):
    if channels:
        img = cv.cvtColor(img, channels)
    A=[img,0,0,0]
    H,V,D = [dict()]*3
    E=list()
    for i in range(1,4):
        A[i], (H[i], V[i], D[i]) = pywt.dwt2(A[i-1][1:-1,1:-1],wavelet=mother, mode='symmetric')
        E.append((H[i]**2+V[i]**2+D[i]**2)**0.5)
    return E

In [429]:
import json
from collections import  OrderedDict
class SlicFire:
    def __init__(self, ann_dir, img_dir, H=400, W=600, out_dir='./sliced_imgs', slic_fast=True, slic_risk=False, preproc=None, transform=None):
        if not os.path.isdir(out_dir):
            os.mkdir(out_dir)
        ann_dir = ann_dir[:-1] if ann_dir[-1] == '/' else ann_dir
        img_dir = img_dir[:-1] if img_dir[-1] == '/' else img_dir
        #print(f'{dir_pos}/{patt_pos}')
        img_files, ann_files = glob(f'{img_dir}/*.jpg'), glob(f'{ann_dir}/*.json') 
        res = OrderedDict()
        for i in sorted(ann_files):
            with open(i,'r') as f:
                buff = json.load(f)['fire']
                buff = list(map(lambda x: (int(x)+1 if int(x)>=0 else 0) if x.isnumeric() else 0, buff))
                if all(buff):
                    res[i.replace('.json','')[-3:]] = np.array(buff)-1
                    
        self.seg = list()
        self.transform = transform
        self.preproc   = preproc
        self._len = sum((len(res[i]) for i in res))
        self.out_dir = out_dir
        self.n_rois = OrderedDict()
        if slic_fast:
            if slic_risk:
                self.S = Slic(num_components=100,compactness=30)
            else:
                def S(iim):
                    return Slic(num_components=100,compactness=30).iterate(iim)
                self.S = S
        kk=0
        for i in tqdm(res, desc='data importing'):
            img = cv.cvtColor(cv.imread(f'{img_dir}/{i}.jpg'), cv.COLOR_BGR2RGB)
            mask = self.S(img)
#             mask = cv.resize(mask, (600,400), cv.INTER_NEAREST)
#             img = cv.resize(img, (600,400), cv.INTER_AREA)
            self.n_rois[i] = mask.max()
#             for j in tqdm(range(100), desc='image prepairing'):
            for j in range(self.n_rois[i]):
                x,y =np.where(mask==j)
                try:
                    x0,x1 = x.min(),x.max()
                    y0,y1 = y.min(),y.max()
                    buff_img = img[x0:x1+1,y0:y1+1]
                    buff_mask = (mask[x0:x1+1,y0:y1+1] == j).astype(np.uint8)
                    img2sav = cv.bitwise_and(buff_img,buff_img,mask = buff_mask.astype(np.uint8))
                    cv.imwrite(f'{self.out_dir}/roi_{str(kk).zfill(3)}.jpg', img2sav)
                    kk+=1
                    if j in res[i]:
                        self.seg.append(int(i)*self.n_rois[i]+j)
                except Exception as E:
                    plt.imshow(mark_boundaries(img,mask,color=(219,112,147), outline_color=(0,0,0)))
                    print('Image',i, 'has ',mask.max(), ' rois')
                    print(x,y,i,j)
                    #plt.imshow(img2sav)
                    #plt.imshow(buff_mask)
                    raise E
                
    def __len__(self):
        return self._len

    def __getitem__(self, idx):
        
        img = cv.imread(f'{self.out_dir}/roi_{str(idx).zfill(3)}.jpg')    
        img = cv.resize(img, (32,32), cv.INTER_AREA)
        
        respond = 1 if idx in self.seg else 0
        if self.transform:
            sample = self.transform(image=img)
            img = sample['image']
            
            
        if  self.preproc:
            img = self.preproc(image=img)['image']

        return (to_tensor(img),get_wv_feature_pyr(img)), respond, 

In [430]:
nngg = list()
for i in range(1000):
    nngg.append(cv.imread(f'./sliced_imgs/roi_{str(i).zfill(3)}.jpg').shape)
print('minima dim : {} * {}'.format(np.array(nngg).T[0].min(), np.array(nngg).T[1].min()))

AttributeError: 'NoneType' object has no attribute 'shape'

Значит, сделаем 32 * 32

In [431]:
plt.imshow((cv.resize(cv.imread(f'./sliced_imgs/roi_000.jpg'), (32,32), cv.INTER_AREA)))

error: OpenCV(4.2.0) /io/opencv/modules/imgproc/src/resize.cpp:4045: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
plt.imshow(np.moveaxis(a[0][0][0], 0,-1).astype(np.uint8))

In [ ]:
import torch
import albumentations as albu
from  torch.utils.data import random_split, DataLoader
from IPython.display import clear_output
from glob import glob
from torchvision import models
from tqdm.notebook import trange, tqdm
import plotly.graph_objects as go
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import time
import copy
import os

In [432]:
def get_augmentation():
    train_transform = [
        
        albu.HorizontalFlip(p=0.5),
        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=(-20,20), shift_limit=0.1, p=1, border_mode=0),
        #albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        #albu.RandomCrop(height=320, width=320, always_apply=True),
        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),
        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),
        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),
        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
        albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]
    return albu.Compose(train_transform)

def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(416, 544)
    ]
    return albu.Compose(test_transform)


def to_tensor(x):
    return x.transpose(2, 0, 1).astype('float32')


def get_preproc(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor),
    ]
    return albu.Compose(_transform)

Just checking if everything is gonna crash

In [440]:
def train_model(model,dataloaders,dataset_sizes, device, criterion, optimizer, num_epochs=20, policy=True):
    if not os.path.isdir('./models'):
        os.mkdir('./models')
    #Being nice to not damage trained state
    optimizer.param_groups[0]['lr'] = 0.1
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

            
    for epoch in trange(num_epochs, desc='EPOCH'):
        if device.type == 'cuda':
            print('MEM |', end='\t')
            print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB', end='\t')
            print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB', end='\n')
        # Each epoch has a training and validation phase
        stats=dict()
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                


                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            stats[f'{phase} acc'] = epoch_acc
            stats[f'{phase} loss'] = epoch_loss
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model, './models/wundervaflya300.pth')
                print('Model saved!')
            if policy and epoch == 5:
                optimizer.param_groups[0]['lr'] = 1e-3
            if policy and epoch == 15:
                optimizer.param_groups[0]['lr'] = 1e-4
        
        if epoch==0:
            fig = go.Figure()
            fig.update_xaxes(range=[0, num_epochs-1])
            for s in stats:
                fig.add_trace(go.Scatter(x=list(), y=list(),
                                mode='lines+markers',name=s))
        
        clear_output(wait=True)
        
        
        for j,s in enumerate(stats):
#             print(stats[s])
#             print(type(stats[s]))
            
            fig['data'][j]['x']+=(epoch,)
            fig['data'][j]['y']+=(float(stats[s]) ,)

        
        fig.show()
        
        
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [434]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

main_dataset = SlicFire('../input/ann-slic/ann', '../input/img-slic/img',transform =get_augmentation())

tr_len = int(0.8*len(main_dataset))
val_len = int(0.15*len(main_dataset))
test_len = len(main_dataset) - tr_len - val_len
train_dataset, val_dataset, test_dataset = random_split(main_dataset,(tr_len,val_len,test_len ) )

image_datasets= {
                    'train': train_dataset,
                    'test': test_dataset,
                    'val':  val_dataset
                }


dataloaders = {x: DataLoader(image_datasets[x], batch_size=10,shuffle=True, num_workers=10) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

cpu


In [441]:
model_ft =  WvTNet3000(32,32)

model_ft = model_ft.to(DEVICE)

#It isn't greate to double our parameters in the last layer, but it's quite tiny, so just leave what is working, BCELoss is similar except \
#mentioned duplicating
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters())

model_ft = train_model(model_ft, dataloaders,dataset_sizes, DEVICE, criterion, optimizer_ft,20, policy=False)

KeyboardInterrupt: 

In [442]:
best_model = torch.load('./models/wundervaflya300.pth')

In [443]:
best_model.eval()

WvTNet3000(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(17, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(33, 63, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4_0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool4): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc4): Linear(in_features=256, out_features=2048, bias=True)
  (dropout4): Dropout(p=0.3, inplace=False)
  (fc5): Linear(in_features=2048, out_features=100, bias=True)
  (fc6): Linear(in_features=100, out_features=2, bias=True)
)

In [ ]:
len(image_datasets['test'])

In [444]:

running_corrects=0
inference_time=0

for i,(inputs, labels) in enumerate(tqdm(DataLoader(image_datasets['test'], batch_size=1,shuffle=True, num_workers=1))): 
                since = time.time()
                outputs = best_model.to('cpu')(inputs)
                _, preds = torch.max(outputs, 1)
                end = time.time()
                inference_time+=end-since
                running_corrects += torch.sum(preds == labels.data)


epoch_acc = running_corrects.double() / len(image_datasets['test'])
print(f'Acc: {epoch_acc}, mean cpu inference time: {inference_time/len(image_datasets["test"])}')


Acc: 0.8888888888888888, mean cpu inference time: 0.004128849064862287


In [ ]:
from IPython.display import FileLink
FileLink('./models/wundervaflya300.pth')

Видно,что время все четко